# Solving a polynomial optimization problem

In [1]:
using JuMP, DynamicPolynomials, MomentTools

In [2]:
X  = @polyvar x1 x2

e1 = x1^2-2
e2 = (x2^2-3)*(x1*x2-2)

p1 = x1
p2 = 2-x2;

We are looking for the points with maximal $x_1$ in the set $e_{1}=e_{2}=0$ such that $p_1\geq 0$, $p_2\geq 0$. 

We solve a SDP relaxation of order $d=4$, where the variables of the underlying convex optimization problem are the moments of order $\le 2d$ in the variables $x_1, x_2$.

In [3]:
using CSDP; optimizer = CSDP.Optimizer
v, M = maximize(x1, [e1, e2], [p1,p2], X, 4, optimizer) 
v

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 6.79e-01 Pobj:  6.1372532e-01 Ad: 6.09e-01 Dobj: -6.8260554e+01 
Iter:  2 Ap: 6.88e-01 Pobj:  1.3077492e+00 Ad: 6.48e-01 Dobj: -9.8470824e+00 
Iter:  3 Ap: 8.37e-01 Pobj:  1.3876892e+00 Ad: 7.83e-01 Dobj:  3.2799189e+01 
Iter:  4 Ap: 7.93e-01 Pobj:  1.4087264e+00 Ad: 8.48e-01 Dobj:  1.8157022e+01 
Iter:  5 Ap: 7.35e-01 Pobj:  1.4126332e+00 Ad: 7.29e-01 Dobj:  9.9917865e+00 
Iter:  6 Ap: 8.77e-01 Pobj:  1.4140585e+00 Ad: 7.21e-01 Dobj:  4.8674622e+00 
Iter:  7 Ap: 6.60e-01 Pobj:  1.4141480e+00 Ad: 7.73e-01 Dobj:  2.5743975e+00 
Iter:  8 Ap: 8.56e-01 Pobj:  1.4142059e+00 Ad: 6.82e-01 Dobj:  1.9576867e+00 
Iter:  9 Ap: 6.93e-01 Pobj:  1.4142101e+00 Ad: 5.68e-01 Dobj:  1.7044330e+00 
Iter: 10 Ap: 5.92e-01 Pobj:  1.4142131e+00 Ad: 6.47e-01 Dobj:  1.5440699e+00 
Iter: 11 Ap: 7.11e-01 Pobj:  1.4142132e+00 Ad: 6.96e-01 Dobj:  1.4691982e+00 
Iter: 12 Ap: 1.00e+00 Pobj:  1.4142134e+00 Ad: 8.69e-

1.414213562494914

The output of the function `maximize` is the optimal value `v` and the optimization model `M`. 

The points which reach the optimal value, can be obtained as follows: 

In [4]:
Xi = getminimizers(M)

2×3 Array{Float64,2}:
  1.41421  1.41421  1.41421
 -1.73205  1.41421  1.73201

Each column of this matrix represents a point. It is an $n\times r$  matrix, where $n$ is the number of their coordinates and $r$ is the number of points.

The weighted sum of Dirac measures associated to the optimal moment sequence can be obtained as follows:

In [5]:
w, Xi = getmeasure(M)

([0.010202656600391963, 0.9859089764203877, 0.0038883674070634765], [1.4142135311334165 1.414213491864442 1.4142134297724813; -1.7320510527503163 1.4142133569292503 1.7320053065575105])

`w` is the vector of weights and `Xi` is the matrix of points, that is support of the measure $\mu=\sum_i \omega_i \delta_{\Xi_i}$.

Here is another way to solve it, describing it as a Polynomial Optimization Problem, using the function `optimize`:

In [6]:
pop = [(x1, "sup"), (e1,"=0"),(e2 ,"=0"),(p1,">=0"),(p2,">=0")]

5-element Array{Tuple{AbstractPolynomialLike{Int64},String},1}:
 (x1, "sup")                     
 (x1² - 2, "=0")                 
 (x1x2³ - 3x1x2 - 2x2² + 6, "=0")
 (x1, ">=0")                     
 (-x2 + 2, ">=0")                

In [7]:
v, M = optimize(pop, X, 4, optimizer)

CSDP 6.2.0
This is a pure primal feasibility problem.
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 6.78e-01 Pobj:  0.0000000e+00 Ad: 6.09e-01 Dobj: -6.8886589e+01 
Iter:  2 Ap: 6.88e-01 Pobj:  0.0000000e+00 Ad: 6.48e-01 Dobj: -1.0829721e+01 
Iter:  3 Ap: 8.37e-01 Pobj:  0.0000000e+00 Ad: 7.83e-01 Dobj:  3.1442596e+01 
Iter:  4 Ap: 7.93e-01 Pobj:  0.0000000e+00 Ad: 8.48e-01 Dobj:  1.6757728e+01 
Iter:  5 Ap: 7.35e-01 Pobj:  0.0000000e+00 Ad: 7.29e-01 Dobj:  8.5780493e+00 
Iter:  6 Ap: 8.77e-01 Pobj:  0.0000000e+00 Ad: 7.21e-01 Dobj:  3.4538249e+00 
Iter:  7 Ap: 6.60e-01 Pobj:  0.0000000e+00 Ad: 7.72e-01 Dobj:  1.1643979e+00 
Iter:  8 Ap: 8.56e-01 Pobj:  0.0000000e+00 Ad: 6.82e-01 Dobj:  5.4579226e-01 
Iter:  9 Ap: 6.86e-01 Pobj:  0.0000000e+00 Ad: 5.71e-01 Dobj:  2.9000700e-01 
Iter: 10 Ap: 5.83e-01 Pobj:  0.0000000e+00 Ad: 6.34e-01 Dobj:  1.3445549e-01 
Iter: 11 Ap: 7.36e-01 Pobj:  0.0000000e+00 Ad: 6.77e-01 Dobj:  6.0112773e-02 
Iter: 12 A

(0.0, 
A MOMent program with:
A JuMP Model
Feasibility problem with:
Variables: 45
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 44 constraints
`Array{GenericAffExpr{Float64,VariableRef},1}`-in-`MathOptInterface.PositiveSemidefiniteConeSquare`: 3 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: CSDP
Names registered in the model: basis, degree, index, moments, monomials, nu, variables, y)

In [8]:
getminimizers(M)

2×3 Array{Float64,2}:
  1.41421  1.41421  1.41421
 -1.73208  1.41251  1.73005